# sensPy Tutorial: Getting Started
This notebook provides a basic tutorial for the `sensPy` package, the Python port of the R package `sensR`.

## 1. Installation
First, let's install `sensPy` directly from its GitHub repository.

In [ ]:
# Ensure you have internet connectivity in your Colab environment for this step
!pip install git+https://github.com/aigorahub/sensPy.git

## 2. Imports
Now, let's import the necessary modules from `sensPy` and other common libraries.

In [ ]:
import numpy as np
import pandas as pd # Optional, for data handling
import senspy.models as spm
import senspy.discrimination as spd
import senspy.power as spp

# Attempt to import and print version (if available)
try:
    from senspy.utils import version # Corrected to use 'version' from utils
    print(f"sensPy version: {version()}")
except ImportError:
    try:
        import senspy
        print(f"sensPy version: {senspy.__version__}")
    except (ImportError, AttributeError):
        print("sensPy version information not found. Ensure package is installed correctly.")

## 3. Core Models API Examples
sensPy primarily uses an object-oriented API for its statistical models. You first create a model instance, then `fit()` it to your data, and then you can get a `summary()` or `confint()` (confidence intervals).

### 3.1. Beta-Binomial Model (`BetaBinomial`)
Used for overdispersed binomial data.

In [ ]:
x_bb = np.array([20, 25])
n_bb = np.array([50, 50])

bb_model = spm.BetaBinomial()
bb_model.fit(x_bb, n_bb, corrected=False)

print("--- BetaBinomial Summary ---")
print(bb_model.summary())
conf_intervals_bb = bb_model.confint(parm=['alpha', 'beta'], method_ci='profile') # Changed method to method_ci
print("\n--- BetaBinomial Confidence Intervals (Profile) ---")
print(conf_intervals_bb)
print(f"\nFitted alpha: {bb_model.alpha:.4f}, beta: {bb_model.beta:.4f}")

### 3.2. General Discrimination Model (`DiscriminationModel`)
Used for various discrimination methods like 2-AFC, Triangle, etc. The `spd.discrim()` function is a wrapper around this.

In [ ]:
disc_model = spm.DiscriminationModel()
disc_model.fit(correct=75, total=100, method="2afc")

print("--- DiscriminationModel Summary (2AFC) ---")
print(disc_model.summary())
ci_disc_profile = disc_model.confint(method_ci='profile')
print("\n--- DiscriminationModel Confidence Intervals (Profile for d-prime) ---")
print(ci_disc_profile)
print(f"\nFitted d-prime: {disc_model.dprime:.4f}")

### 3.3. Two-Alternative Choice Model with Bias (`TwoACModel`)
Estimates sensitivity (delta) and bias (tau).

In [ ]:
twoac_model = spm.TwoACModel()
twoac_model.fit(hits=70, false_alarms=15, n_signal_trials=100, n_noise_trials=100)

print("--- TwoACModel Summary ---")
print(twoac_model.summary())
ci_twoac_profile = twoac_model.confint(method_ci='profile')
print("\n--- TwoACModel Confidence Intervals (Profile) ---")
print(ci_twoac_profile)
print(f"\nFitted delta: {twoac_model.delta:.4f}, tau: {twoac_model.tau:.4f}")

### 3.4. Degree of Difference Model (`DoDModel`)

In [ ]:
same_counts_dod = np.array([10, 20, 70])
diff_counts_dod = np.array([70, 20, 10])

dod_model = spm.DoDModel()
dod_model.fit(same_counts_dod, diff_counts_dod)

print("--- DoDModel Summary ---")
print(dod_model.summary())
# Note: CIs for tpar are returned from Wald; Profile CIs are more complex for DoD
ci_dod_profile = dod_model.confint(parm=['d_prime', 'tpar_0', 'tpar_1'], method_ci='profile')
print("\n--- DoDModel Confidence Intervals (Profile) ---")
print(ci_dod_profile)
print(f"\nFitted d-prime: {dod_model.d_prime:.4f}, tau: {dod_model.tau}")

### 3.5. Same-Different Model (`SameDifferentModel`)

In [ ]:
sd_model = spm.SameDifferentModel()
sd_model.fit(nsamesame=70, ndiffsame=30, nsamediff=25, ndiffdiff=75)

print("--- SameDifferentModel Summary ---")
print(sd_model.summary())
ci_sd_profile = sd_model.confint(method_ci='profile')
print("\n--- SameDifferentModel Confidence Intervals (Profile) ---")
print(ci_sd_profile)
print(f"\nFitted delta: {sd_model.delta:.4f}, tau: {sd_model.tau:.4f}")

## 4. Functional Wrappers
For quick analyses or users more comfortable with a functional style, `senspy.discrimination` provides wrappers that return dictionaries.

In [ ]:
discrim_dict_result = spd.discrim(correct=60, total=90, method="triangle")
print("--- spd.discrim (functional wrapper) output ---")
for key, value in discrim_dict_result.items():
    if isinstance(value, float): print(f"  {key}: {value:.4f}")
    else: print(f"  {key}: {value}")

## 5. Power and Sample Size (`senspy.power`)
Calculate statistical power or required sample size.

In [ ]:
# Calculate power for a discrimination task (exact binomial method)
power_res_exact = spp.power_discrim(
    d_prime_alt=0.8, 
    n_trials=50, 
    method="2afc", 
    d_prime_null=0.0, 
    alpha_level=0.05, 
    alternative="greater"
)
print("--- Power Calculation (Exact Method) ---")
print(f"Power: {power_res_exact.power:.4f}")
print(f"Method Type: {power_res_exact.method_type}")
print(f"Details: {power_res_exact.details}")

# Calculate required sample size (exact binomial method)
n_required_exact = spp.sample_size_discrim(
    d_prime_alt=0.8, 
    target_power=0.8, 
    method="2afc", 
    d_prime_null=0.0, 
    alpha_level=0.05, 
    alternative="greater"
)
print("\n--- Sample Size Calculation (Exact Method) ---")
print(f"Required N: {n_required_exact}")

# Power calculation using normal approximation
power_res_approx = spp.power_discrim_normal_approx(
    d_prime_alt=0.8, 
    n_trials=50, 
    method="2afc",
    d_prime_null=0.0,
    alpha_level=0.05,
    alternative="greater"
)
print("\n--- Power Calculation (Normal Approximation) ---")
if power_res_approx and isinstance(power_res_approx, spp.PowerResult): # Check it's PowerResult
    print(f"Power: {power_res_approx.power:.4f}")
    print(f"Details: {power_res_approx.details}")

## 6. Other Utilities
`senspy.discrimination` also contains link functions (`psyfun`, `psyinv`, `psyderiv`, `rescale`), ROC utilities (`SDT`, `AUC`), and d-prime hypothesis tests (`dprime_test`, `dprime_compare`).

In [ ]:
pc_val = spd.psyfun(dprime=1.2, method="triangle")
print(f"Proportion correct for d'=1.2 (triangle): {pc_val:.4f}")

sdt_counts = np.array([[50,30,15,5], [5,15,30,50]]) # Noise, Signal
sdt_output = spd.SDT(sdt_counts)
print("\n--- SDT Output (zFA, zH, d-prime per criterion) ---")
print(sdt_output)

## Conclusion
This notebook covered the basic installation and usage of key features in `sensPy`. For more details, please refer to the official documentation and the `guide_to_switch.md` if you are coming from `sensR`.